<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_model_load_work_transformer_demo_fp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model environmenmt

In [ ]:
!pip install colab-env -q

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Mounted at /content/gdrive


In [ ]:
!pip install transformers datasets torch geopy -q

In [4]:
!pip show transformers
print()
!pip show datasets
print()
!pip show torch

Name: transformers
Version: 4.51.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers

Name: datasets
Version: 3.5.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tq

## training

In [5]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [6]:
from transformers import AutoTokenizer
tokenizer_name = 'gpt2'
#tokenizer_name = 'meta-llama/Meta-Llama-3-8B'
#tokenizer_name = 'mistralai/Mistral-7B-Instruct-v0.3'
pad_token = '<pad>'

# --- Load Tokenizer and Add Pad Token ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': pad_token})
# Ensure pad_token_id is within the vocabulary size
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")
flight_plan_dataset['train'][7]

README.md:   0%|          | 0.00/12.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/381k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'input': 'Calculate the waypoints from TPA to ZRH. Departure: 2024-12-04, Aircraft: Boeing 767, Weather: Partly Cloudy',
 'label': 7,
 'distance': 7851.313536308213,
 'distance_category': 'long',
 'waypoints': [[27.9791649, -82.5349276153517],
  [43.138360187535824, -11.666039390889935],
  [43.63996560781222, -9.321045623867192],
  [42.74001242229881, -13.528306002908437],
  [36.0068887801934, -45.005503686758225],
  [44.65896776666469, -4.557234041099491],
  [42.244966187576004, -15.842635728798143],
  [40.04132875759056, -26.14458987284717],
  [47.463388050000006, 8.553366081422642]],
 'waypoint_names': ['TPA',
  'Waypoint',
  'Waypoint',
  'Waypoint',
  'Waypoint',
  'Waypoint',
  'Waypoint',
  'Waypoint',
  'ZRH']}

In [21]:
flight_plan_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'distance', 'distance_category', 'waypoints', 'waypoint_names'],
        num_rows: 2000
    })
})

In [8]:
def convert_token_ids_to_coords(token_ids, tokenizer):
    """
    Converts token IDs to waypoint coordinates.

    Args:
        token_ids (torch.Tensor): Token IDs representing waypoints.
        tokenizer: The tokenizer used for the model.

    Returns:
        torch.Tensor: Waypoint coordinates as a tensor.
    """

    # 1. Decode token IDs to text, clipping to valid vocabulary range
    decoded_text = tokenizer.decode(token_ids[0].clamp(0, tokenizer.vocab_size - 1))  # Assuming batch size of 1 for simplicity

    # *** Print the decoded text ***
    print(f"Decoded Text: {decoded_text}")

    # 2. Extract waypoint coordinates from text using regular expressions
    # Assuming the waypoints are in the format "latitude, longitude" (e.g., 34.0522, -118.2437)
    # Adjust the regex to match the actual waypoint format in your data
    # Example regex: r"(-?\d+\.\d+),\s*(-?\d+\.\d+)"
    waypoints_text = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", decoded_text)  # Updated regex

    # *** Print the extracted waypoints text ***
    #print(f"Waypoints Text: {waypoints_text}")

    # 3. Convert waypoint text to coordinate tensors
    coordinates = []
    for lat, lon in waypoints_text:  # Unpack lat and lon directly
        coordinates.append([float(lat), float(lon)])

    # 4. Convert to PyTorch tensor
    coordinates_tensor = torch.tensor(coordinates)

    return coordinates_tensor

In [9]:
# Cell 45: custom_loss_function

import torch.nn as nn # Make sure you have imported nn

def custom_loss_function(predicted_waypoints, target_waypoints, label, tokenizer):
    """
    Calculates the custom loss for waypoint coordinates.

    Args:
        predicted_waypoints (torch.Tensor): Model's predicted waypoints (coordinates).
        target_waypoints (torch.Tensor): Target waypoints (token IDs).
        label (int): The expected number of waypoints for the flight route.
        tokenizer: The tokenizer used for the model.

    Returns:
        torch.Tensor: The total loss.
    """

    # 1. Convert target_waypoints (token IDs) to coordinates
    target_coords = convert_token_ids_to_coords(target_waypoints, tokenizer)

    # 2. Handle empty target_coords
    if target_coords.nelement() == 0:  # Check if target_coords is empty
        # You can return a default loss or handle the case as appropriate
        #print("Target coordinates are empty. Returning a default loss.")
        return torch.tensor(0.0, requires_grad=True, device=predicted_waypoints.device)  # Ensure requires_grad=True for the default loss

    # 3. Calculate the distance-based loss (e.g., Mean Squared Error)
    # Reshape predicted_waypoints to match target_coords shape if necessary
    predicted_waypoints = predicted_waypoints.view(-1, 2)  # Reshape to (batch_size * seq_len, 2)

    # Ensure target_coords has the expected shape
    if target_coords.dim() == 1:  # If target_coords is 1D (only one coordinate pair)
        target_coords = target_coords.unsqueeze(0)  # Add an extra dimension

    target_coords = target_coords.view(-1, 2)  # Reshape to (batch_size * seq_len, 2)

    # *** Print shapes before loss calculation ***
    print(f"Predicted Waypoints Shape: {predicted_waypoints.shape}")
    print(f"Target Coords Shape: {target_coords.shape}")

    standard_loss = nn.MSELoss()(predicted_waypoints, target_coords)

    # 4. Calculate the waypoint count penalty
    # (You might need to adjust this part based on how label is used)
    # Assuming 'label' represents the target number of waypoints
    count_penalty = abs(predicted_waypoints.shape[0] - label.shape[0])  # Compare number of waypoints

    # *** Print individual losses ***
    print(f"Standard Loss: {standard_loss.item()}")
    print(f"Count Penalty: {count_penalty}")

    # 5. Combine the losses
    total_loss = standard_loss + 0.1 * count_penalty

    return total_loss

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

batch_size = 4
sequence_length = 128  # Adjusted based on tokenize_function

def tokenize_function(examples):
    # Directly use waypoint numbers as labels
    examples["labels"] = examples["label"]
    tokenized_output = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length
    )

    # Instead of assigning labels directly, shift them for causal LM
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()

    # Replace input_ids corresponding to pad_token with -100 in labels
    tokenized_output['labels'] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # --- Ensure input_ids and labels are within vocabulary range before padding ---
    tokenized_output['input_ids'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['input_ids']
    ]

    # --- Replace out-of-vocabulary tokens in labels with unk_token_id ---
    tokenized_output['labels'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # Handle case where labels are empty (could happen due to errors in data)
    for i, label_list in enumerate(tokenized_output['labels']):
        if len(label_list) == 0:
            # Replace with a single UNK token or handle as appropriate
            tokenized_output['labels'][i] = [tokenizer.unk_token_id]

    return tokenized_output


# Load dataset
dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Tokenize, format, and split dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["input", "label"])
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# --- Split the 'train' dataset within the DatasetDict ---
train_testvalid = tokenized_datasets['train'].train_test_split(test_size=0.2, seed=42)

# Create new DatasetDict with the splits
tokenized_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': train_testvalid['test']
})

# Further split the 'test' set into validation and test
test_valid = tokenized_datasets['test'].train_test_split(test_size=0.5, seed=42)

# Update the DatasetDict
tokenized_datasets = DatasetDict({
    'train': tokenized_datasets['train'],
    'validation': test_valid['test'], # Renamed to 'validation'
    'test': test_valid['train'] # Renamed to 'test'
})


train_dataset = tokenized_datasets["train"]
testvalid_dataset = tokenized_datasets["test"]  # Assuming you still need this
eval_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

small_train_dataset = train_dataset.shuffle(seed=42).select(range(1600))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(200))

# 2. Data Loading
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['distance', 'distance_category', 'waypoints', 'waypoint_names', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['distance', 'distance_category', 'waypoints', 'waypoint_names', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
    test: Dataset({
        features: ['distance', 'distance_category', 'waypoints', 'waypoint_names', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [ ]:
!pip install transformers datasets torch geopy -q

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# Configuration
tokenizer_name = "gpt2"
batch_size = 4
sequence_length = 128  # Adjusted based on tokenize_function
embedding_dimension = 256
num_heads = 8
feed_forward_dimension = 1024
num_encoder_layers = 6
dropout_probability = 0.3
learning_rate = 2e-5
num_epochs = 1

# Load Tokenizer and Add Pad Token (if needed)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})

# Ensure pad_token_id is within the vocabulary size
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1  # Clip to maximum vocabulary index


# 1. Tokenize and Format
def tokenize_function(examples):
    # Directly use waypoint numbers as labels
    examples["labels"] = examples["label"]
    tokenized_output = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length
    )

    # Instead of assigning labels directly, shift them for causal LM
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()

    # Replace input_ids corresponding to pad_token with -100 in labels
    tokenized_output['labels'] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # --- Ensure input_ids are within vocabulary range before padding ---
    tokenized_output['input_ids'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['input_ids']
    ]

    # --- Additional Checks ---
    # Ensure that labels are within the vocabulary range as well
    tokenized_output['labels'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # Handle case where labels are empty (could happen due to errors in data)
    for i, label_list in enumerate(tokenized_output['labels']):
        if len(label_list) == 0:
            # Replace with a single UNK token or handle as appropriate
            tokenized_output['labels'][i] = [tokenizer.unk_token_id]

    return tokenized_output


# Load dataset
dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Tokenize, format, and split dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["input", "label"])
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# --- Split the 'train' dataset within the DatasetDict ---
train_testvalid = tokenized_datasets['train'].train_test_split(test_size=0.2, seed=42)

# Create new DatasetDict with the splits
tokenized_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': train_testvalid['test']
})

# Further split the 'test' set into validation and test
test_valid = tokenized_datasets['test'].train_test_split(test_size=0.5, seed=42)

# Update the DatasetDict
tokenized_datasets = DatasetDict({
    'train': tokenized_datasets['train'],
    'validation': test_valid['test'], # Renamed to 'validation'
    'test': test_valid['train'] # Renamed to 'test'
})


train_dataset = tokenized_datasets["train"]
testvalid_dataset = tokenized_datasets["test"]  # Assuming you still need this
eval_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

small_train_dataset = train_dataset.shuffle(seed=42).select(range(1600))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(200))

# 2. Data Loading
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

# 3. Model
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_layers, block_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, ff_hidden_dim, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.linear_out = nn.Linear(embed_dim, vocab_size)  # Output layer for token prediction
        self.block_size = block_size
        self.dropout = nn.Dropout(dropout)

        # --- Initialize weights to handle potential out-of-vocabulary tokens ---
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data) * 0.02  # Example initialization


        # *** Define waypoint_coords_layer (Example) ***
        # Replace with your desired layer for converting output to waypoint coordinates
        self.waypoint_coords_layer = nn.Linear(vocab_size, 2)  # Example: Output 2 values (lat, lon) # Change is here





    def forward(self, input_ids, attention_mask):
        # Ensure input_ids are within vocabulary range
        input_ids = input_ids.clamp(0, tokenizer.vocab_size - 1)

        # Create positional embeddings - Adjusted to use batch size
        batch_size = input_ids.shape[0]
        positions = torch.arange(0, input_ids.size(1), dtype=torch.long, device=input_ids.device)
        positions = positions.unsqueeze(0).repeat(batch_size, 1) # Repeat for each item in batch
        pos_embeddings = self.pos_embedding(positions)

        # Combine embeddings
        embeddings = self.dropout(self.embedding(input_ids) + pos_embeddings)

        # Create src_key_padding_mask with correct shape
        src_key_padding_mask = (input_ids == tokenizer.pad_token_id).transpose(0, 1)  # Transpose here
        src_key_padding_mask = src_key_padding_mask.to(input_ids.device)

        #6. Pass to encoder

        encoder_output  = self.encoder(embeddings, src_key_padding_mask=src_key_padding_mask)

        # Output layer
        #output = self.linear_out(encoder_output)
        #return output

        # Output layer
        output = self.linear_out(encoder_output)
        # --- Apply waypoint_coords_layer to the output ---
        # Reshape output to (batch_size * sequence_length, vocab_size) # Change is here
        output = output.view(-1, output.size(-1))
        waypoint_coords = self.waypoint_coords_layer(output)

        return waypoint_coords  # Return latitude and longitude


model = SimpleTransformer(
    vocab_size=tokenizer.vocab_size,
    embed_dim=embedding_dimension,
    num_heads=num_heads,
    ff_hidden_dim=feed_forward_dimension,
    num_layers=num_encoder_layers,
    block_size=sequence_length,
    dropout=dropout_probability
)

# 4. Loss and Optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training and Evaluation Loop

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1} - Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        waypoint_counts = batch['labels'].to(device)  # Use 'labels' for waypoint count

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        # Calculate loss with custom loss function
        # Pass the tokenizer to custom_loss_function
        loss = custom_loss_function(outputs, labels, waypoint_counts, tokenizer)


        # Check for NaN loss
        if torch.isnan(loss):
            print("Loss is NaN. Skipping this batch.")
            continue

        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average Training Loss: {avg_train_loss:.4f}")

    # --- Evaluation ---
    model.eval()
    total_eval_loss = 0

    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc=f"Epoch {epoch + 1} - Evaluation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            waypoint_counts = batch['labels'].to(device)  # Use 'labels' for waypoint count

            outputs = model(input_ids, attention_mask)

            # Calculate loss with custom loss function
            # *** Pass the tokenizer to custom_loss_function ***
            loss = custom_loss_function(outputs, labels, waypoint_counts, tokenizer)

            total_eval_loss += loss.item()

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch + 1} - Average Evaluation Loss: {avg_eval_loss:.4f}")

## Model Deployment To HF

In [13]:
!pip install --upgrade huggingface_hub -q
from huggingface_hub import notebook_login, create_repo, upload_file

In [14]:
from huggingface_hub import notebook_login, create_repo, upload_file

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# --- Automatically generate README.md ---
readme_content = f"""
# Flight Plan Transformer

This repository contains a Transformer model trained to generate flight plan waypoints based on textual input. The model was trained using the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets.

## Model Description

The model is a decoder-only Transformer architecture implemented using PyTorch and the `transformers` library. It takes textual input describing the flight (e.g., "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy") and predicts a sequence of waypoints representing the flight path.

## Intended Use

This model can be used to:

- **Generate flight plans:** Given flight details, the model can predict waypoints for the route.
- **Flight planning assistance:** It can serve as a tool to aid in flight planning, providing suggestions for waypoints.

## Limitations

- **Accuracy:** The model's accuracy is limited by the quality and coverage of the training data. It may not be perfectly accurate in all scenarios.
- **Real-world applicability:** This model is a research prototype and should not be used for actual flight navigation without thorough validation and safety checks.

## How to Use

1. **Install the necessary libraries:**
Use code with caution
bash pip install transformers datasets torch

2. **Load the model and tokenizer:**
Use code with caution
python from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("{repo_name}") model = AutoModelForSeq2SeqLM.from_pretrained("{repo_name}")

3. **Generate waypoints:**
Use code with caution
python def generate_flight_plan(model, tokenizer, start_text, max_new_tokens=200):

# ... (Your generation code from the original script) ...
4. **Example usage:**
Use code with caution
python query = "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy" generated_plan = generate_flight_plan(model, tokenizer, start_text=query, max_new_tokens=256) print(generated_plan)

## Training Data

The model was trained on the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets. This dataset contains flight plans with corresponding waypoints.

## Evaluation

The model's performance was evaluated on a held-out portion of the training data. (Include specific metrics or evaluation results if available)

## Acknowledgements

- Thanks to Hugging Face for providing the `transformers` and `datasets` libraries.
- Thanks to frankmorales2020 for creating and sharing the flight plan dataset.

## Contact
Frank Morales, BEng, MEng, SMIEEE
Boeing Associate Technical Fellow
Linkedin: https://www.linkedin.com/in/frank-morales1964/
"""

# Write README.md to file
with open("README.md", "w") as f:
    f.write(readme_content)

In [15]:
import os
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
token=access_token_write

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.get_token_permission(token=access_token_write)
repo_id = 'frankmorales2020/FlightPlan_Transformer_LLM'
api.delete_repo(repo_id=repo_id)

In [17]:
from huggingface_hub import HfApi, HfFolder
import os
import shutil
import json

# --- Push to Hugging Face Hub ---

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"
create_repo(repo_name, private=False, exist_ok=True)

# --- Push the model directly from memory ---
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=repo_name,
    repo_type="model",
)

# Save the model and tokenizer to a temporary directory
model_save_dir = "flight_plan_transformer_model"
os.makedirs(model_save_dir, exist_ok=True)

# 1. Create a configuration dictionary for your model
config = {
    "vocab_size": tokenizer.vocab_size,
    "embed_dim": embedding_dimension,  # Use your embedding dimension variable
    "num_heads": num_heads,  # Use your num_heads variable
    "ff_hidden_dim": feed_forward_dimension,  # Use your feed_forward_dimension variable
    "num_layers": num_encoder_layers,  # Use your num_encoder_layers variable
    #"block_size": block_size,  # Use your block_size variable
    "block_size": sequence_length,  # Use sequence_length as block_size
    "dropout": dropout_probability  # Use your dropout_probability variable
}

# 2. Save the configuration to config.json
with open(os.path.join(model_save_dir, "config.json"), "w") as f:
    json.dump(config, f)

# Save the model's state_dict instead of using save_pretrained
torch.save(model.state_dict(), os.path.join(model_save_dir, 'pytorch_model.bin'))
tokenizer.save_pretrained(model_save_dir) # This is fine as tokenizer is from Hugging Face


# Upload the model files, including config.json and pytorch_model.bin
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "config.json"),
    path_in_repo="config.json",
    repo_id=repo_name,
)
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "pytorch_model.bin"),
    path_in_repo="pytorch_model.bin",
    repo_id=repo_name,
)

# Upload other files (e.g., tokenizer files)
for filename in os.listdir(model_save_dir):
    if filename not in ["config.json", "pytorch_model.bin"]:  # Exclude already uploaded files
        upload_file(
            path_or_fileobj=os.path.join(model_save_dir, filename),
            path_in_repo=filename,
            repo_id=repo_name,
        )


# Remove the temporary directory (optional)
shutil.rmtree(model_save_dir)

print(f"Model pushed to Hugging Face Hub: {repo_name}")

pytorch_model.bin:   0%|          | 0.00/123M [00:00<?, ?B/s]

Model pushed to Hugging Face Hub: frankmorales2020/FlightPlan_Transformer_LLM


## Model Evaluation

In [18]:
!pip install datasets -q

In [19]:
import torch
import torch.nn as nn
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_layers, block_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, ff_hidden_dim, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.linear_out = nn.Linear(embed_dim, vocab_size)  # Output layer for token prediction
        self.block_size = block_size
        self.dropout = nn.Dropout(dropout)

        # --- Initialize weights to handle potential out-of-vocabulary tokens ---
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data) * 0.02  # Example initialization

    def forward(self, input_ids, attention_mask):
        # Ensure input_ids are within vocabulary range
        input_ids = input_ids.clamp(0, tokenizer.vocab_size - 1)

        # Create positional embeddings - Adjusted to use batch size
        batch_size = input_ids.shape[0]
        positions = torch.arange(0, input_ids.size(1), dtype=torch.long, device=input_ids.device)
        positions = positions.unsqueeze(0).repeat(batch_size, 1) # Repeat for each item in batch
        pos_embeddings = self.pos_embedding(positions)

        # Combine embeddings
        embeddings = self.dropout(self.embedding(input_ids) + pos_embeddings)

        # Create src_key_padding_mask with correct shape
        src_key_padding_mask = (input_ids == tokenizer.pad_token_id).transpose(0, 1)  # Transpose here
        src_key_padding_mask = src_key_padding_mask.to(input_ids.device)

        #6. Pass to encoder

        encoder_output  = self.encoder(embeddings, src_key_padding_mask=src_key_padding_mask)

        # Output layer
        output = self.linear_out(encoder_output)

        return output


In [20]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
import json
from huggingface_hub import hf_hub_download

# --- Updated Model Loading ---
pretrained_model_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# 1. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
#model=AutoModelForCausalLM.from_pretrained(pretrained_model_name)

#
# 2. Load configuration using hf_hub_download
config_path = hf_hub_download(repo_id=pretrained_model_name, filename="config.json")
with open(config_path, 'r') as f:
    config_dict = json.load(f)

# *** Add max_waypoints to the config dictionary or load it from elsewhere ***
config_dict['max_waypoints'] = 10  # Replace with your actual max_waypoints value

# 3. Instantiate your SimpleTransformer class using the loaded configuration
model = SimpleTransformer(
    vocab_size=config_dict['vocab_size'],
    embed_dim=config_dict['embed_dim'],
    num_heads=config_dict['num_heads'],
    ff_hidden_dim=config_dict['ff_hidden_dim'],
    num_layers=config_dict['num_layers'],
    block_size=config_dict['block_size'],
    dropout=config_dict['dropout'],
)

# 4. Load the model weights using hf_hub_download
model_path = hf_hub_download(repo_id=pretrained_model_name, filename="pytorch_model.bin")
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

tokenizer_config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/123M [00:00<?, ?B/s]

RuntimeError: Error(s) in loading state_dict for SimpleTransformer:
	Unexpected key(s) in state_dict: "waypoint_coords_layer.weight", "waypoint_coords_layer.bias". 

In [ ]:
print(model)

In [ ]:
# --- Load and Preprocess Flight Plan Data ---
from datasets import load_dataset, DatasetDict
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Split into train and validation sets
flight_plan_dataset = flight_plan_dataset['train'].train_test_split(test_size=0.2, seed=42)
flight_plan_dataset = DatasetDict({
    'train': flight_plan_dataset['train'],
    'validation': flight_plan_dataset['test']
})

In [ ]:
flight_plan_dataset

In [ ]:
flight_data=flight_plan_dataset['validation'][100]
print(flight_data['input'])

In [ ]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [ ]:
# Cell 62: generate_flight_plan

def generate_flight_plan(model, tokenizer, query, number_of_waypoints, device='cuda'):
    model.eval()
    model.to(device)

    # Start with the initial query tokens
    start_tokens = tokenizer(query, return_tensors='pt', padding=True, truncation=True).to(device)

    # *** Print start tokens for debugging ***
    print(f"Start Tokens: {start_tokens}")
    print(f"Length of start_tokens: {len(start_tokens)}")
    print(f"Length of start_tokens['input_ids']: {len(start_tokens['input_ids'])}")
    print(number_of_waypoints)

    with torch.no_grad():
        generated_waypoints = []
        while len(generated_waypoints) < number_of_waypoints:  # Stopping condition

            # Access 'input_ids' and 'attention_mask' directly from start_tokens dictionary
            generated_waypoints = model(start_tokens['input_ids'], start_tokens['attention_mask'])
            print(f"Generated waypoints: {generated_waypoints}")
            #print(f"Length of generated_waypoints: {len(generated_waypoints)}")
            #print(f"Length of generated_waypoints[0]: {len(generated_waypoints[0])}")
            #print(f"Length of generated_waypoints[0][0]: {len(generated_waypoints[0][0])}")

    # Post-processing: Limit the final list (Optional)
    #generated_waypoints = generated_waypoints[:number_of_waypoints]

    return generated_waypoints

In [ ]:
def generate_flight_plan(model, tokenizer, query, number_of_waypoints, device='cuda'):
    model.eval()
    model.to(device)

    # Start with the initial query tokens
    # start_tokens should be a dictionary containing both 'input_ids' and 'attention_mask'
    start_tokens = tokenizer(query, return_tensors='pt', padding=True, truncation=True).to(device)
    print(f"Start tokens: {start_tokens}")
    print(f"Length of start_tokens: {len(start_tokens)}")
    print(f"Length of start_tokens['input_ids']: {len(start_tokens['input_ids'])}")
    print(number_of_waypoints)

    with torch.no_grad():
        generated_waypoints = []
        while len(generated_waypoints) < number_of_waypoints:  # Stopping condition

            # Access 'input_ids' and 'attention_mask' directly from start_tokens dictionary
            generated_waypoints = model(start_tokens['input_ids'], start_tokens['attention_mask'])
            #print(f"Generated waypoints: {generated_waypoints}")
            #print(f"Length of generated_waypoints: {len(generated_waypoints)}")
            #print(f"Length of generated_waypoints[0]: {len(generated_waypoints[0])}")
            #print(f"Length of generated_waypoints[0][0]: {len(generated_waypoints[0][0])}")

    # Post-processing: Limit the final list (Optional)
    generated_waypoints = generated_waypoints[:number_of_waypoints]

    return generated_waypoints

In [ ]:
flight_plan_dataset

In [ ]:
for i in range(1):
    flight_data=flight_plan_dataset['validation'][i]
    query=flight_data['input']
    print(f"Query: {query}")
    print(f"Waypints list: {flight_data['waypoints']}")
    number_of_waypoints=len(flight_data['waypoints'])
    print(f"Number of wapoints: {number_of_waypoints}")
    print('\n')
    generated_plan = generate_flight_plan(model, tokenizer, query, number_of_waypoints, device=device)
    print(f"\nGenerated waypoint list: {generated_plan}")